# Finding Similar Wikipedia Articles with MinHashLSH

This notebook demonstrates how to use the MinHashLSH (Locality Sensitive Hashing) algorithm in PySpark to find similar documents in a text-based dataset.

**Credits**

Adapted from [Detecting Abuse at Scale: Locality Sensitive Hashing at Uber Engineering](https://www.uber.com/blog/lsh/)

**Compatibility**

| Platform                     | Compatible | Recommended | Notes                                                                                                                                                         |
| ---------------------------- | ---------- | ----------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Local (e.g., M1 MacBook)** | ✅ Yes      | ✅ Yes       | -  |
| **Google Colab**             | ✅ Yes      | ✅ Yes       |  - |
| **Midway3 Login Node**       | ✅ Yes      | ✅ Yes       | - |
| **Midway3 Compute Node**     | ✅ Yes      | ✅ Yes      | -  |


## 1. Load Raw Data

First, we need a Spark session. In a real-world scenario on a computing cluster like Amazon EMR, you would load data from a distributed file system like HDFS. For this local demonstration, we'll create a sample DataFrame directly.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import BooleanType
from pyspark.ml.feature import Tokenizer, CountVectorizer, MinHashLSH
from pyspark.ml.linalg import Vectors

In [ ]:
# Initialize Spark Session
spark = SparkSession.builder.appName("MinHashLSH_Tutorial").getOrCreate()

# For this notebook, we'll create a sample DataFrame to simulate the raw data
sample_data = [
    (
        "United States",
        "The United States of America (U.S.A. or USA), commonly known as the United States (U.S. or US) or America, is a country primarily located in North America.",
    ),
    (
        "USA",
        "The U.S. is a federal republic and a representative democracy with three separate branches of government.",
    ),
    (
        "Canada",
        "Canada is a country in North America. Its ten provinces and three territories extend from the Atlantic to the Pacific and northward into the Arctic Ocean.",
    ),
    (
        "Germany",
        "Germany is a country in Central Europe. It is the second-most populous country in Europe after Russia.",
    ),
    (
        "United Kingdom",
        "The United Kingdom, made up of England, Scotland, Wales and Northern Ireland, is an island nation in northwestern Europe.",
    ),
    (
        "America",
        "The term America (or the Americas) refers to the landmasses of North and South America. The United States is often referred to as America.",
    ),
]
df_sample = spark.createDataFrame(sample_data, ["title", "content"])

print("Sample Wikipedia articles:")
df_sample.show(truncate=False)

Sample Wikipedia articles:
+--------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|title         |content                                                                                                                                                    |
+--------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|United States |The United States of America (U.S.A. or USA), commonly known as the United States (U.S. or US) or America, is a country primarily located in North America.|
|USA           |The U.S. is a federal republic and a representative democracy with three separate branches of government.                                                  |
|Canada        |Canada is a country in North America. Its ten provinces and three territories extend from th

---


## 2. Prepare Feature Vectors

**MinHashLSH** works with feature vectors. We need to convert the text content of each article into a numerical vector. This process involves two main steps:

1.  **Tokenization**: Splitting the article content into individual words.
2.  **Vectorization**: Converting the list of words into a feature vector of word counts using `CountVectorizer`.


In [ ]:
# Tokenize the article content
tokenizer = Tokenizer(inputCol="content", outputCol="words")
words_df = tokenizer.transform(df_sample)

# Vectorize the words for each article
vocab_size = 1000
cv = CountVectorizer(
    inputCol="words", outputCol="features", vocabSize=vocab_size, minDF=1.0
)
cv_model = cv.fit(words_df)

# Filter out any articles that might have resulted in empty vectors
vectorized_df = (
    cv_model.transform(words_df)
    .filter(col("features").isNotNull())
    .select(col("title"), col("features"))
)
print("Articles converted to feature vectors:")
vectorized_df.show(truncate=False)

Articles converted to feature vectors:
+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|title         |features                                                                                                                                                          |
+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|United States |(76,[0,1,2,3,5,6,7,8,9,10,12,13,15,30,31,34,37,48,60,65,68,73],[2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])         |
|USA           |(76,[0,1,2,4,6,16,18,26,27,40,41,54,55,59,69],[1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                                                      |
|Canada        |(76,[0,1,2,3,4,5,7,11,12,16,20,24,32,38,39,44

---


## 3. Fit and Query the LSH Model

Now we can fit our MinHashLSH model to the vectorized data. The `numHashTables` parameter allows us to balance between performance and accuracy.

PySpark's [`MinHashLSH`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.MinHashLSH.html) is a powerful tool for finding approximate nearest neighbors in large datasets, especially for comparing sets of items, like the words in documents.
- It takes a DataFrame as **input**, where one column (specified by `setInputCol`) contains feature vectors, which are typically sparse vectors representing sets (e.g., word counts from a document).
- After fitting the `MinHashLSH` model, it produces a new DataFrame with an **output** column (specified by `setOutputCol`) containing the hash values for each input vector.

The main argument, `numHashTables`, is the equivalent of `bands` discussed in the **illustrated guide**: more hash tables increase the accuracy of finding similar items but also increase computation time.

In [ ]:
# Initialize and fit the MinHashLSH model
minhash = MinHashLSH(inputCol="features", outputCol="hashValues", numHashTables=3)
lsh_model = minhash.fit(vectorized_df)

# Feature Transformation: Show the hash values for each article
print("Transformed data with hash values:")
hashed_df = lsh_model.transform(vectorized_df)

hashed_df.show(truncate=False)

Transformed data with hash values:
+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------+
|title         |features                                                                                                                                                          |hashValues                                     |
+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------+
|United States |(76,[0,1,2,3,5,6,7,8,9,10,12,13,15,30,31,34,37,48,60,65,68,73],[2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])         |[[4.4752084E7], [8.6080663E7], [3.074331E7]]   |
|USA           |(76,[0,1,2,4,6,16,18,26,27,40,41,54,5

### Query the Model

Once you have a fitted [`MinHashLSHModel`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.MinHashLSHModel.html), you can use it to find similar items in two primary ways:

* **`approxNearestNeighbors`**: This function is like a "search" operation. You provide a single feature vector (the "key") that you want to find matches for, and an integer **k**, and it will efficiently search through the entire dataset to return the top **k** items that are most similar to your key. The similarity is measured by the Jaccard distance between the items' feature vectors.

* **`approxSimilarityJoin`**: This function is used to find all pairs of similar items *within* a dataset (or between two different datasets) that meet a certain similarity level. You provide a **threshold** (a value between 0 and 1 for the Jaccard distance), and it returns all pairs of items from the datasets whose distance is less than or equal to that threshold. It's incredibly efficient for tasks like large-scale document deduplication or finding all related product pairs in a massive catalog.

> ⚠: The `approxSimilarityJoin` uses Jaccard distance (`1 - similarity`) instead of similarity as threshold, so you need to set the threshold accordingly.

### Approximate Nearest Neighbor Search

We can now use the trained LSH model to find the most similar articles to a given key. For this example, we'll create a key vector representing the phrase "united states" and search for the top 3 most similar articles.

In [ ]:
# Create a key vector for the query "united states"
key_vector_indices = [
    cv_model.vocabulary.index(word)
    for word in ["united", "states"]
    if word in cv_model.vocabulary
]
search_key = Vectors.sparse(
    vocab_size, sorted(list(set([(i, 1.0) for i in key_vector_indices])))
)

# Find the 3 nearest neighbors
num_neighbors = 3
print(f"Approximate nearest neighbors for 'united states':")
lsh_model.approxNearestNeighbors(hashed_df, search_key, num_neighbors).show(
    truncate=False
)

Approximate nearest neighbors for 'united states':
+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------+------------------+
|title         |features                                                                                                                                                 |hashValues                                  |distCol           |
+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------+------------------+
|America       |(76,[0,1,4,6,7,8,9,11,12,13,15,17,19,28,33,45,53,62,66],[4.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                   |[[9536300.0], [2.28624684E8], [3.074331E7]] |0.8947368421052632|
|United S

### Approximate Similarity Join

To find all pairs of articles in the dataset that are similar to each other, we can use an approximate similarity join. We set a `threshold` for the Jaccard distance to determine which pairs are considered "similar".


In [ ]:
# Perform a self-join to find similar pairs of articles within the dataset
distance_threshold = 0.8  # Jaccard distance threshold
print(f"Pairs of articles with a Jaccard distance <= {distance_threshold}:")
lsh_model.approxSimilarityJoin(
    hashed_df, hashed_df, distance_threshold, distCol="JaccardDistance"
).filter("JaccardDistance > 0").select(
    col("datasetA.title").alias("Title A"),
    col("datasetB.title").alias("Title B"),
    col("JaccardDistance"),
).show(
    truncate=False
)

Pairs of articles with a Jaccard distance <= 0.8:
+-------------+-------------+---------------+
|Title A      |Title B      |JaccardDistance|
+-------------+-------------+---------------+
|America      |United States|0.71875        |
|United States|America      |0.71875        |
+-------------+-------------+---------------+



---


## 4. Performance and efficiency

As noted in the original tutorial, LSH provides a significant performance benefit over brute-force methods. The trade-off between speed and accuracy, tunable via the `numHashTables` parameter, makes LSH a powerful tool for large-scale similarity detection tasks. 🚀

- **Approximate Nearest Neighbor Search**: Can be significantly faster (e.g., 2x) than a full scan.
- **Approximate Similarity Join**: Can be 3x-5x faster than a full cross-join and filter.

This speedup is achieved while maintaining high accuracy, making LSH a practical and efficient solution for finding similar items in massive datasets.


<img src="./assets/lsh_perf.avif" alt="Performance Comparison of LSH vs. Brute Force" width="800">

In [ ]:
# Stop the Spark Session
spark.stop()